In [1]:
# %reset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
from sklearn import metrics
import math
from sklearn.decomposition import PCA
import pickle
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import robust_scale
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from warnings import simplefilter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [2]:
from os import listdir
df=shuffle(pd.read_parquet('src_data/'+listdir('src_data/')[0])).sample(320000)

for f in listdir('src_data/')[1:]:
    if f.split('.')[-1]!='pqt': continue
    df=pd.concat([df, pd.read_parquet('src_data/'+f).loc[df.index]], axis=1)
df=df[df.describe().columns].convert_dtypes('Float64')
del listdir

In [3]:
df_attributed = df['is_attributed'].astype('int')
df=df.drop('is_attributed', axis=1)
df=(df-df.quantile(0.02))/(df.quantile(0.95)-df.quantile(0.01))-0.5
df=df.apply(lambda col: col.apply(lambda v: v if np.isscalar(v)
            and 0.5>=v>=-0.5 else np.sign(v)*math.log(abs(v*8)+1,5)/2))
df = df[~((df<df.quantile(0.01)) | (df>df.quantile(0.99))).any(axis=1)]
df=(df-df.min())/(df.max()-df.min())
df=df[df.median().sort_values().index]

df['is_attributed']=df_attributed.loc[df.index]

display(df.describe())
display(df_attributed.describe())
df_source=df

,device_os_svd_2,device_app_svd_3,device_channel_svd_3,device_channel_svd_1,device_os_svd_1,device_app_svd_1,app_device_svd_1,os_device_svd_1,app_os_svd_1,os_app_svd_1,...,device_os_svd_4,device_channel_svd_2,device_app_svd_4,device_os_svd_3,device_count,device_app_svd_2,device_os_svd_0,device_channel_svd_0,device_app_svd_0,is_attributed
count,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,2.537570e+05,2.537570e+05,253757.000000,253757.000000,253757.000000,...,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000,253757.000000
mean,0.041118,0.042393,0.043208,0.036201,0.034081,3.407401e-02,3.401285e-02,0.031242,0.034189,0.030429,...,0.894663,0.972336,0.972817,0.943056,0.941853,0.972925,0.939889,0.941086,0.940213,0.157631
std,0.175550,0.178532,0.180575,0.181550,0.181413,1.814141e-01,1.812625e-01,0.166282,0.181230,0.163004,...,0.146435,0.158561,0.158504,0.225758,0.228564,0.158496,0.236299,0.231574,0.235048,0.364395
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2.314094e-11,2.123501e-11,0.000013,0.000061,0.000065,...,0.915358,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,2.314094e-11,3.873968e-11,0.000023,0.000099,0.000189,...,0.915358,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,2.314094e-11,5.639789e-11,0.000026,0.000162,0.000224,...,0.915358,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


count    320000.000000
mean          0.199044
std           0.399282
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: is_attributed, dtype: float64

In [11]:
df=df_source
df_target=df['is_attributed'];df=df.drop('is_attributed', axis=1)
pca_module = PCA()
pca_module.fit(df)
pca_comp=pca_module.transform(df)
pca_comp=pd.DataFrame(pca_comp).set_index(df.index)#.iloc[:,:32]
df=pca_comp
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
count,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,...,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05
mean,-7.571447e-17,-4.257539e-17,4.312140e-18,3.861326e-17,-1.058434e-17,2.969497e-17,-5.989395e-17,-4.250538e-17,-9.089096e-17,4.722354e-17,...,2.510004e-17,4.170230e-16,-2.638980e-18,-1.542333e-18,2.072803e-18,6.824342e-18,9.567759e-19,3.862930e-18,-2.320087e-18,-6.230255e-18
std,1.037138e+00,8.741859e-01,7.397236e-01,6.436235e-01,6.153795e-01,5.817543e-01,5.729090e-01,5.209250e-01,4.568158e-01,3.595626e-01,...,1.011847e-15,4.962407e-17,1.527743e-16,7.961912e-17,1.603500e-16,2.079954e-16,1.259405e-16,1.120439e-16,6.849517e-17,5.175018e-16
min,-1.731410e+00,-1.538654e+00,-1.162652e+00,-1.317297e+00,-1.265410e+00,-8.429111e-01,-1.244605e+00,-1.067371e+00,-1.086681e+00,-1.022311e+00,...,-1.714951e-15,-3.138091e-16,-1.029447e-15,-1.075350e-15,-9.883421e-16,-1.419755e-15,-5.052823e-16,-5.337415e-16,-3.537006e-16,-2.284189e-15
25%,-7.223774e-01,-5.443663e-01,-4.830957e-01,-4.204048e-01,-4.503977e-01,-3.242544e-01,-4.395494e-01,-3.385943e-01,-3.250936e-01,-2.160490e-01,...,-8.753589e-17,4.043664e-16,-7.274695e-17,-5.880166e-17,-7.847965e-17,-9.052348e-17,-6.232458e-17,-1.593600e-17,-3.069689e-17,-3.139763e-16
50%,3.542787e-01,-1.963659e-01,-2.019186e-01,-1.373168e-01,-1.828514e-01,-1.780317e-01,-8.801774e-02,-3.830221e-02,-4.641890e-02,3.077354e-04,...,3.736420e-17,4.251831e-16,1.051977e-17,3.648387e-18,1.172597e-17,-1.766509e-17,-1.028287e-17,-2.058209e-18,3.997579e-18,5.212797e-18
75%,6.986189e-01,4.781840e-01,1.155238e-01,3.602293e-01,3.050447e-01,2.796412e-02,4.934691e-01,2.343247e-01,3.795762e-01,2.229384e-01,...,1.414476e-16,4.459998e-16,7.990871e-17,5.222064e-17,1.019316e-16,1.627461e-16,4.175883e-17,1.875847e-17,3.175315e-17,3.244019e-16
max,2.904399e+00,3.369386e+00,2.910846e+00,2.628750e+00,1.703004e+00,2.267549e+00,1.302074e+00,3.180085e+00,2.592405e+00,1.228948e+00,...,3.326252e-14,5.708998e-16,6.830487e-16,5.899849e-16,2.019668e-15,8.100718e-16,8.536094e-16,5.199582e-16,3.164210e-16,7.506591e-15


In [13]:
X, y = df.values, df_target.values
model = SelectFromModel(LogisticRegression(C=1, penalty="l1",
                        solver='liblinear', random_state=7).fit(X, y), prefit=True)
selected_features = pd.DataFrame(
    model.inverse_transform(model.transform(X)),index=df.index, columns=df.columns)
to_be_removed = selected_features.columns[selected_features.var() == 0]
df=df.drop(to_be_removed, axis=1)
df_L1=df
df_L1.describe()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,62,65
count,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,...,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05
mean,-7.571447e-17,-4.257539e-17,4.312140e-18,3.861326e-17,-1.058434e-17,2.969497e-17,-5.989395e-17,-4.250538e-17,-9.089096e-17,4.722354e-17,...,4.989412e-18,3.344621e-17,-1.133394e-16,1.210573e-16,5.189838e-17,7.062005e-17,4.437269e-18,-5.231380e-17,-1.881727e-17,6.240156e-18
std,1.037138e+00,8.741859e-01,7.397236e-01,6.436235e-01,6.153795e-01,5.817543e-01,5.729090e-01,5.209250e-01,4.568158e-01,3.595626e-01,...,1.990104e-02,1.343140e-02,1.163198e-02,9.010251e-03,8.102932e-03,7.015439e-03,6.618327e-03,5.218810e-03,4.758338e-03,1.916538e-03
min,-1.731410e+00,-1.538654e+00,-1.162652e+00,-1.317297e+00,-1.265410e+00,-8.429111e-01,-1.244605e+00,-1.067371e+00,-1.086681e+00,-1.022311e+00,...,-1.309633e-01,-9.235914e-02,-5.044168e-02,-4.600067e-02,-3.375120e-02,-3.501111e-02,-4.987765e-02,-1.812416e-02,-3.064098e-02,-1.701810e-02
25%,-7.223774e-01,-5.443663e-01,-4.830957e-01,-4.204048e-01,-4.503977e-01,-3.242544e-01,-4.395494e-01,-3.385943e-01,-3.250936e-01,-2.160490e-01,...,-5.409256e-03,-4.148888e-03,-6.803566e-03,-4.300216e-03,-1.758406e-03,-3.699929e-03,-2.466012e-03,-2.043869e-03,-1.834491e-03,-3.338619e-04
50%,3.542787e-01,-1.963659e-01,-2.019186e-01,-1.373168e-01,-1.828514e-01,-1.780317e-01,-8.801774e-02,-3.830221e-02,-4.641890e-02,3.077354e-04,...,-6.577685e-04,-1.165870e-04,-5.622907e-06,3.731796e-04,4.459736e-04,2.906529e-04,-3.320795e-04,-3.565152e-04,-5.950972e-05,5.686164e-05
75%,6.986189e-01,4.781840e-01,1.155238e-01,3.602293e-01,3.050447e-01,2.796412e-02,4.934691e-01,2.343247e-01,3.795762e-01,2.229384e-01,...,4.972987e-03,1.714155e-03,7.267643e-03,4.366496e-03,2.933820e-03,2.877366e-03,2.809515e-03,2.511833e-03,1.539411e-03,4.014022e-04
max,2.904399e+00,3.369386e+00,2.910846e+00,2.628750e+00,1.703004e+00,2.267549e+00,1.302074e+00,3.180085e+00,2.592405e+00,1.228948e+00,...,3.095991e-01,1.045693e-01,6.827440e-02,5.224934e-02,4.053291e-02,5.635270e-02,6.369975e-02,3.917342e-02,3.377294e-02,1.823572e-02


In [16]:
df=df_L1
selector = SelectKBest(f_classif,k=55)
X_new = selector.fit_transform(df, df_target)
selected_features = pd.DataFrame(selector.inverse_transform(X_new),
                                index=df.index, columns=df.columns)
to_be_removed = selected_features.columns[selected_features.var() == 0]
df=df.drop(to_be_removed, axis=1)
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,57,58,59,60,62,65
count,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,...,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05,2.537570e+05
mean,-7.571447e-17,-4.257539e-17,4.312140e-18,3.861326e-17,-1.058434e-17,2.969497e-17,-5.989395e-17,-4.250538e-17,-9.089096e-17,4.722354e-17,...,1.218040e-17,4.989412e-18,3.344621e-17,-1.133394e-16,5.189838e-17,7.062005e-17,4.437269e-18,-5.231380e-17,-1.881727e-17,6.240156e-18
std,1.037138e+00,8.741859e-01,7.397236e-01,6.436235e-01,6.153795e-01,5.817543e-01,5.729090e-01,5.209250e-01,4.568158e-01,3.595626e-01,...,2.311468e-02,1.990104e-02,1.343140e-02,1.163198e-02,8.102932e-03,7.015439e-03,6.618327e-03,5.218810e-03,4.758338e-03,1.916538e-03
min,-1.731410e+00,-1.538654e+00,-1.162652e+00,-1.317297e+00,-1.265410e+00,-8.429111e-01,-1.244605e+00,-1.067371e+00,-1.086681e+00,-1.022311e+00,...,-7.804090e-02,-1.309633e-01,-9.235914e-02,-5.044168e-02,-3.375120e-02,-3.501111e-02,-4.987765e-02,-1.812416e-02,-3.064098e-02,-1.701810e-02
25%,-7.223774e-01,-5.443663e-01,-4.830957e-01,-4.204048e-01,-4.503977e-01,-3.242544e-01,-4.395494e-01,-3.385943e-01,-3.250936e-01,-2.160490e-01,...,-7.138139e-03,-5.409256e-03,-4.148888e-03,-6.803566e-03,-1.758406e-03,-3.699929e-03,-2.466012e-03,-2.043869e-03,-1.834491e-03,-3.338619e-04
50%,3.542787e-01,-1.963659e-01,-2.019186e-01,-1.373168e-01,-1.828514e-01,-1.780317e-01,-8.801774e-02,-3.830221e-02,-4.641890e-02,3.077354e-04,...,-1.949518e-03,-6.577685e-04,-1.165870e-04,-5.622907e-06,4.459736e-04,2.906529e-04,-3.320795e-04,-3.565152e-04,-5.950972e-05,5.686164e-05
75%,6.986189e-01,4.781840e-01,1.155238e-01,3.602293e-01,3.050447e-01,2.796412e-02,4.934691e-01,2.343247e-01,3.795762e-01,2.229384e-01,...,2.313124e-03,4.972987e-03,1.714155e-03,7.267643e-03,2.933820e-03,2.877366e-03,2.809515e-03,2.511833e-03,1.539411e-03,4.014022e-04
max,2.904399e+00,3.369386e+00,2.910846e+00,2.628750e+00,1.703004e+00,2.267549e+00,1.302074e+00,3.180085e+00,2.592405e+00,1.228948e+00,...,9.789266e-02,3.095991e-01,1.045693e-01,6.827440e-02,4.053291e-02,5.635270e-02,6.369975e-02,3.917342e-02,3.377294e-02,1.823572e-02


In [23]:
pickle.dump(df, open('working_data/initial_changed_data.pickle','wb'))
del SelectFromModel, SelectKBest, X, X_new, df_source, df_L1, df_attributed,\
    f, model, selected_features, selector, to_be_removed, y

In [ ]:
# **************************
df=shuffle(pickle.load(open('working_data/initial_changed_data.pickle','rb')).sample(2500))
not_atr=df[df_target==0]
df=shuffle(df.drop(not_atr.sample(len(not_atr)//10).index))
x_train, x_test, y_train, y_test = train_test_split(df,
                        df_target.loc[df.index], test_size=0.2, random_state=42)
pipe=make_pipeline(
    SMOTE(),
    RandomForestClassifier()#class_weight={0:1, 1:2})
)
print("Best parameters : %s" % GridSearchCV(
        estimator=pipe,
        param_grid={
            'smote__sampling_strategy': np.linspace(0.005, 1, 7),
            'randomforestclassifier__class_weight': [{0:1, 1:i} for i in np.linspace(0.25, 2.25, 7)]
        },
        scoring='f1',
    ).fit(x_train.values, y_train.values).best_params_)

In [51]:
# **************************
def run_pipe(pipe, save_name=None, sample_size=None):
    df=shuffle(pickle.load(open('working_data/initial_changed_data.pickle','rb')))
    if sample_size is not None: df=df.sample(sample_size)
    not_atr=df[df_target==0]
    df=shuffle(df.drop(not_atr.sample(len(not_atr)//10).index))
    x_train, x_test, y_train, y_test = train_test_split(df,
                            df_target.loc[df.index], test_size=0.2, random_state=42)

    pipe.fit(x_train, y_train)

    y_predict=pipe.predict(x_test)
    display(pd.crosstab(y_predict, y_test, rownames=['Predicted'], colnames=['Actual']))
    print("Accuracy = " , accuracy_score(y_test, y_predict))
    print("Precision = " ,precision_score(y_test, y_predict))
    print("Recall = " ,recall_score(y_test, y_predict))
    print("F1 Score = " ,f1_score(y_test, y_predict))

    if save_name is not None: pickle.dump(pipe, open('working_data/'+save_name+'.pickle', 'wb'))
    return pipe

In [63]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy=0.34),
    LogisticRegression(class_weight={0: 1, 1: 0.75})
), 'LogisticRegression')

C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,38211,1724
1,353,6189


Accuracy =  0.955311229210147
Precision =  0.9460409660654234
Recall =  0.7821306710476431
F1 Score =  0.8563126945693532


In [68]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy= 0.5),
    SVC(class_weight={0: 1, 1: 0.9})
), 'SVC', 50000)

C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,7562,342
1,57,1196


Accuracy =  0.9564267773288195
Precision =  0.9545091779728652
Recall =  0.7776332899869961
F1 Score =  0.8570404872805447


In [69]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy= 0.34),
    KNeighborsClassifier()
),'KNeighborsClassifier', 100000)

C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,14672,617
1,490,2534


Accuracy =  0.9395511385354666
Precision =  0.8379629629629629
Recall =  0.8041891463027611
F1 Score =  0.8207287449392714


In [70]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy= 0.34),
    DecisionTreeClassifier(class_weight={0: 1, 1: 1.25})
),'DecisionTreeClassifier', 100000)

C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,14307,667
1,907,2434


Accuracy =  0.914059514059514
Precision =  0.7285243938940437
Recall =  0.7849080941631732
F1 Score =  0.7556659422539584


In [71]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy= 0.34),
    RandomForestClassifier(class_weight={0: 1, 1: 1.25})
),'RandomForestClassifier', 100000)

C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,14977,648
1,200,2492


Accuracy =  0.9537042092045641
Precision =  0.9257057949479941
Recall =  0.7936305732484077
F1 Score =  0.8545953360768177


In [72]:
#*************
pipe = run_pipe(make_pipeline(
    SMOTE(sampling_strategy= 0.4),
    GaussianNB()
),'GaussianNB', 100000)


C:\Users\No1\AppData\Local\Temp/ipykernel_15620/685942475.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_atr=df[df_target==0]


Actual,0,1
Predicted,,
0,14708,782
1,566,2258


Accuracy =  0.9263951075679808
Precision =  0.7995750708215298
Recall =  0.7427631578947368
F1 Score =  0.7701227830832197
